<a href="https://colab.research.google.com/github/kd365/faafall22/blob/hill/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NTSB Cleaning Notebook

## Steps:
1. Import libraries and raw data
2. Initial column drops
3. Cleaning steps that wil apply to entire dataframe
4. Cleaning each column

**Cleaning Plan:**
Hunter:
- cols 0-22
Kathleen:
- col 23-45
Eric:
- col 46-68
Chi:
- col 69-85


### 1. Import libraries

In [1]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

Saving NTSB_for_cleaning_total.csv to NTSB_for_cleaning_total.csv


In [ ]:
import re

In [2]:
# setting pandas display options
pd.set_option('display.max_rows', 150)

# reading in NTSB csv file
ntsb_raw = pd.read_csv("NTSB_for_cleaning_total.csv", encoding='latin-1', low_memory=False)
ntsb_raw = ntsb_raw.drop(['engines_ev_id', 'events_ev_id', 'Unnamed: 0'], axis=1)
# view the dataframe info
ntsb_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104120 entries, 0 to 104119
Data columns (total 86 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   aircraft_ev_id         104120 non-null  object 
 1   far_part               104120 non-null  object 
 2   flight_plan_activated  45517 non-null   object 
 3   damage                 103326 non-null  object 
 4   acft_make              104108 non-null  object 
 5   acft_model             104096 non-null  object 
 6   cert_max_gr_wt         93685 non-null   float64
 7   num_eng                102746 non-null  float64
 8   type_last_insp         99030 non-null   object 
 9   date_last_insp         83764 non-null   object 
 10  afm_hrs_last_insp      58888 non-null   float64
 11  afm_hrs                84919 non-null   float64
 12  type_fly               100532 non-null  object 
 13  dprt_apt_id            85958 non-null   object 
 14  dprt_city              88866 non-nul

### 2. Initial Drop

In [3]:
# drop all records that do not have at least 50 columns worth of data in them
ntsb_raw = ntsb_raw.dropna(thresh=50)

In [ ]:
# view data
ntsb_raw.head()

,aircraft_ev_id,far_part,flight_plan_activated,damage,acft_make,acft_model,cert_max_gr_wt,num_eng,type_last_insp,date_last_insp,...,inj_tot_t,wx_cond_basic,Cause_Factor,crew_no,crew_category,crew_age,crew_sex,med_certf,pilot_flying,crew_inj_level
0,20001204X00000,135,NaN,SUBS,Cessna,207,3800.0,1.0,100H,12/16/1998,...,1.0,VMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
1,20001204X00004,135,NaN,SUBS,Cessna,207,3800.0,1.0,AAIP,12/14/1998,...,NaN,VMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2,20001204X00005,091,NaN,SUBS,Piper,PA-22-160,1840.0,1.0,ANNL,3/14/1998,...,NaN,VMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,20001204X00006,091,NaN,DEST,Beech,300,14100.0,2.0,AAIP,1/14/1999,...,2.0,IMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,20001204X00007,091,NaN,DEST,Piper,PA-28-181,2550.0,1.0,UNK,NaN,...,1.0,VMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN


In [4]:
# drop any columns that have more than 40% null values
total_rows = ntsb_raw.shape[0]

def drop_cols(df, df_size):
    # create a list of columns to drop
    drop = []
    # iterate over each column
    for x in df.columns:
        #determine if the ratio of nulls is greater than 30%
        ratio = (df[x].isna().sum()) / df_size
        if ratio > 0.5:
            drop.append(x)
    # drop the columns in the drop list
    df.drop(labels=drop, axis=1)
    print('Cols dropped from df:', drop)
    return df

ntsb_dropped_cols = drop_cols(ntsb_raw, total_rows)      


Cols dropped from df: ['flight_plan_activated', 'dest_same_local', 'phase_flt_spec', 'eng_time_total', 'eng_time_last_insp', 'apt_dir', 'vis_rvr', 'wx_dens_alt', 'wx_int_precip', 'Cause_Factor', 'crew_no', 'crew_category', 'crew_age', 'crew_sex', 'med_certf', 'crew_inj_level']


In [5]:
print(ntsb_dropped_cols.shape)

(104120, 86)


## 3. Cleaning that applies to entire data set

In [6]:
# need to delete duplicate records
ntsb_dropped_cols = ntsb_dropped_cols.drop_duplicates(subset='aircraft_ev_id')

In [7]:
# function to initially clean up all strings in df
def df_string_transform(df):
    ''' 
    force all text data to lowercase and strip left and right side of strings
    in all string columns

    args: dataframe

    returns: dataframe
    '''
    df1 = df.applymap(lambda x: x.lower() if type(x) == str else x)
    df1 = df1.applymap(lambda x: x.strip() if type(x) == str else x)
    return df1

In [8]:
# applying the funcrition from above
ntsb_1 = df_string_transform(ntsb_dropped_cols)

In [9]:
print(ntsb_1.shape)

(61705, 86)


**NOTE**: for section 3 use the dataframe `ntsb_1` for cleaning

### 3. Cleaning Each Column

In [10]:
# function to show all the metrics you will want to see for each column
def col_metrics(df, col='none'):
  print('Unique data: ', df[col].unique())
  print('Value counts: ', df[col].value_counts())
  print('Total nulls: ', df[col].isna().sum())
  print('Data type: ', df[col].dtypes)

### HUNTER'S SECTION

In [ ]:
# column 1: far part
col_metrics(ntsb_1, col='far_part')      

# making a copy of the data set as ntsb_1
ntsb_1 = pd.DataFrame.copy(ntsb_1)

# remove the K and F from part 091
ntsb_1['far_part'] = ntsb_1['far_part'].str.replace('k', '').str.replace('f', '').str.replace('arm', '091')
print(ntsb_1['far_part'].unique())

Unique data:  ['135' '091' '137' '103' 'armf' '091f' '091k' '437']
Value counts:  091     55126
137      3407
135      2988
103       130
091f       35
091k       12
armf        6
437         1
Name: far_part, dtype: int64
Total nulls:  0
Data type:  object
['135' '091' '137' '103' '437']


In [ ]:
# column 2: 'flight_plan_activated'
col_metrics(ntsb_1, 'flight_plan_activated')

# dropping this column, too many nulls, and filing a flight plan is superfluous 
#because you can file in flight or you can use flight following
ntsb_1 = ntsb_1.drop(['flight_plan_activated'], axis=1)

Unique data:  [nan 'y' 'n' 'u']
Value counts:  n    12307
y     3188
u      354
Name: flight_plan_activated, dtype: int64
Total nulls:  45856
Data type:  object


In [ ]:
# column 3: 'damage'
col_metrics(ntsb_1, 'damage')

# change the unk to none
ntsb_1['damage'] = ntsb_1['damage'].str.replace('unk', 'none')
# drop the 491 rows that do not have a damage listing
ntsb_1 = ntsb_1.dropna(subset='damage')
print('final null count:', ntsb_1['damage'].isna().sum())

Unique data:  ['subs' 'dest' 'none' 'minr' 'unk' nan]
Value counts:  subs    46690
dest    13612
minr      963
none      330
unk         9
Name: damage, dtype: int64
Total nulls:  101
Data type:  object
final null count: 0


In [ ]:
# column 4: 'acft_make'
col_metrics(ntsb_1, 'acft_make')

# there are 6 rosw with nulls, will drop those
ntsb_1 = ntsb_1.dropna(subset=['acft_make'])
print(ntsb_1['acft_make'].isna().sum())

Unique data:  ['cessna' 'piper' 'beech' ... 'gerald nelson' 'gonzalez manuel a'
 'harris-runyan']
Value counts:  cessna             21141
piper              11468
beech               4212
bell                1513
mooney              1073
                   ...  
quinn aviation         1
larry robbins          1
polliwagen inc.        1
woolsey                1
harris-runyan          1
Name: acft_make, Length: 5657, dtype: int64
Total nulls:  10
Data type:  object
0


In [ ]:
#if an aircraft make is not in the top 30, it will be categorized as other. 
acft_top_20 = ntsb_1['acft_make'].value_counts()[0:30].index.tolist()

ntsb_1['acft_make'] = ntsb_1['acft_make'].map(lambda x: 'other_make' if x not in acft_top_20 else x)

print(ntsb_1['acft_make'].value_counts())

cessna            21141
other_make        15091
piper             11468
beech              4212
bell               1513
mooney             1073
grumman             922
bellanca            860
robinson            602
hughes              571
schweizer           522
aeronca             487
maule               442
champion            394
air tractor         389
stinson             336
boeing              332
luscombe            327
aero commander      319
taylorcraft         300
north american      293
Name: acft_make, dtype: int64


In [ ]:
# column 5: 'acft_model'
col_metrics(ntsb_1, 'acft_model')

# drop the 5 rows that have nulls
ntsb_1 = ntsb_1.dropna(subset=['acft_model'])

Unique data:  ['207' 'pa-22-160' '300' ... 'rv 7' 'f4u 5' 'skybolt 300']
Value counts:  152            2014
172            1091
172n            967
pa-28-140       810
172m            672
               ... 
m6235             1
j-3br503          1
a-98              1
vega-1            1
skybolt 300       1
Name: acft_model, Length: 8264, dtype: int64
Total nulls:  21
Data type:  object


In [ ]:
# column 5

# remove the letters off the cessna model
pattern = r'\d\d\d\w'
ntsb_1['acft_model'] = ntsb_1['acft_model'].map(lambda x: re.sub(pattern, lambda match: match.group()[:-1], x))
print(ntsb_1['acft_model'].value_counts())

#if an aircraft make is not in the top 30, it will be categorized as other. 
model_top_30 = ntsb_1['acft_model'].value_counts()[0:30].index.tolist()

# change the model names to 'other_model' if not in the top 30
ntsb_1['acft_model'] = ntsb_1['acft_model'].map(lambda x: 'other_model' if x not in model_top_30 else x)

172                  4756
150                  2658
152                  2030
182                  1995
pa-28-140             830
                     ... 
type 43 series 3f       1
7gca                    1
22/20                   1
a152erobat              1
skybolt 300             1
Name: acft_model, Length: 7653, dtype: int64
other_model    38441
172             4756
150             2658
152             2030
182             1995
pa-28-140        830
180              771
g-164            763
t210             629
210              592
206              581
170              543
pa-28-180        498
pa-28-161        486
310              462
pa-28-181        439
pa-18-150        427
pa-38-112        426
177              406
pa-18            397
269              393
a36              359
140              347
185              340
pa-32-300        314
pa-24-250        313
pa-28r-200       292
a188             282
a185             282
421              265
pa-34-200        256
Name: acft_model, 

In [ ]:
# column 6: 'cert_max_gr_wt'
col_metrics(ntsb_1, 'cert_max_gr_wt')
print(ntsb_1['cert_max_gr_wt'].max(), ntsb_1['cert_max_gr_wt'].min())

def weight_bins(df, col):
    avg_wt = sum(list(df[col])) / len(list(df[col]))
    wt_cat = []
    for x in list(df[col]):
        if x <= 12500:
            x = 'small'
            wt_cat.append(x)
        elif x > 12500 and x < 41000:
            x = 'medium'
            wt_cat.append(x)
        elif x == 'nan':
            if avg_wt <=12500:
                x = 'small'
                wt_cat.append(x)
            elif avg_wt > 12500 and x < 41000:
                x = 'medium'
                wt_cat.append(x)
            else:
                x = 'large commuter'
                wt_cat.append(x)
        else:
            x = 'large commuter'
            wt_cat.append(x)
    df[col] = wt_cat
    return df[col]

Unique data:  [ 3800.  1840. 14100. ... 21419.  2497.  2251.]
Value counts:  1600.0     2757
2300.0     2709
1670.0     2301
3600.0     1605
2150.0     1592
           ... 
2256.0        1
4808.0        1
3470.0        1
38500.0       1
2251.0        1
Name: cert_max_gr_wt, Length: 2245, dtype: int64
Total nulls:  3410
Data type:  float64
882000.0 -3100.0


In [ ]:
# use the bin function to assign categories to weight classes
ntsb_1['cert_max_gr_wt'] = weight_bins(ntsb_1, 'cert_max_gr_wt')

In [ ]:
# column 7: 'num_eng'
col_metrics(ntsb_1, 'num_eng')

# convert the float to int and then impute the nulls with the median
ntsb_1['num_eng'] = ntsb_1['num_eng'].

Unique data:  [ 1.  2. nan  3.  4.  0.]
Value counts:  1.0    53194
2.0     7049
0.0      642
4.0       58
3.0       31
Name: num_eng, dtype: int64
Total nulls:  599
Data type:  float64


## KATHLEEN'S SECITON

In [ ]:
ntsb_2 = weight_bins(ntsb_1, 'cert_max_gr_wt')

In [ ]:
ntsb_2['cert_max_gr_wt']

0        14100.0
1         6000.0
2         5300.0
3         5121.0
4         4300.0
          ...   
58683        NaN
58684        NaN
58685        NaN
58686        NaN
58687        NaN
Name: cert_max_gr_wt, Length: 58677, dtype: float64

In [ ]:
ntsb_1.columns

Index(['aircraft_ev_id', 'far_part', 'flight_plan_activated', 'damage',
       'acft_make', 'acft_model', 'cert_max_gr_wt', 'num_eng',
       'type_last_insp', 'date_last_insp', 'afm_hrs_last_insp', 'afm_hrs',
       'type_fly', 'dprt_apt_id', 'dprt_city', 'dprt_state', 'dprt_time',
       'dest_same_local', 'dest_apt_id', 'dest_city', 'dest_state',
       'phase_flt_spec', 'afm_hrs_since', 'rwy_num', 'rwy_len', 'rwy_width',
       'ifr_equipped_cert', 'eng_no', 'eng_type', 'eng_mfgr', 'hp_or_lbs',
       'carb_fuel_injection', 'eng_time_total', 'eng_time_last_insp',
       'ntsb_no', 'ev_type', 'ev_date', 'ev_dow', 'ev_time', 'ev_city',
       'ev_state', 'ev_year', 'ev_month', 'latitude', 'longitude', 'apt_name',
       'ev_nr_apt_id', 'ev_nr_apt_loc', 'apt_dist', 'apt_dir', 'apt_elev',
       'wx_src_iic', 'wx_obs_time', 'wx_obs_dir', 'wx_obs_fac_id',
       'wx_obs_elev', 'wx_obs_dist', 'light_cond', 'sky_cond_nonceil',
       'sky_nonceil_ht', 'sky_ceil_ht', 'sky_cond_ceil', 'vis_

In [ ]:
ntsb_raw = ntsb_raw.loc[:, ~ntsb_raw.columns.str.contains('^Unnamed')]

In [ ]:
print(ntsb_raw['damage'].unique())

In [ ]:
print(ntsb_raw['type_last_insp'].unique())

In [ ]:
ntsb_rv1 = ntsb_raw.dropna(axis=1, how='all')
#removed one column
ntsb_rv1.describe(include='all') 

In [ ]:
ntsb_rv1.info()

In [ ]:

print(ntsb_rv1.dtypes)

In [ ]:
ntsb_rv1.head()

In [ ]:
print(ntsb_rv1[['ev_time']].to_string(index=False)) 

In [ ]:
ntsb_rv1['ev_time'] = pd.to_numeric(ntsb_rv1['ev_time'])

In [ ]:
import numpy as np
ntsb_rv1['ev_time'] = ntsb_rv1['ev_time'].astype(np.int64)

In [ ]:
nan_count = ntsb_rv1['ev_time'].isna().sum()
print(nan_count)

In [ ]:
print((ntsb_rv1['ev_time'] == 'NaN').sum())

In [ ]:
#adding leading zeros for military time format
ntsb_rv1['ev_time'] = ntsb_rv1['ev_time'].apply(lambda x: '{0:0>4}'.format(x))
print(ntsb_rv1[['ev_time']].to_string(index=False)) 


In [ ]:
ntsb_rv1['ev_time'] = pd.to_datetime(ntsb_rv1['ev_time'], format = '%H%m')

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
ntsb_rv1['date_last_insp'] =  pd.to_datetime(ntsb_rv1['date_last_insp'], infer_datetime_format=True)


In [ ]:
#ntsb_rv2= ntsb_rv1.drop(['Aircraft_Key'], axis=1)

In [ ]:
print(ntsb_rv1.dtypes)

In [ ]:
ntsb_rv1['wx_obs_time'] =  pd.to_datetime(ntsb_rv1['wx_obs_time'], format = "%H%M")

In [ ]:
#Walkthrough at https://inria.github.io/scikit-learn-mooc/python_scripts/03_categorical_pipeline_column_transformer.html
#preprocessing different for categorical vs numerical columns

from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(ntsb_raw)
categorical_columns = categorical_columns_selector(ntsb_raw)

In [ ]:
## Starting cleaning, columns 23-45


# function to show all the metrics you will want to see for each column
#def col_metrics(df, col='none'):
#  print('Unique data: ', df[col].unique())
#  print('Value counts: ', df[col].value_counts())
#  print('Total nulls: ', df[col].isna().sum())
#  print('Data type: ', df[col].dtypes)

#work with df ntsb_1

In [13]:
#print(df.columns[2])
col_list = ntsb_1.columns[23:46]
print(col_list)

Index(['rwy_num', 'rwy_len', 'rwy_width', 'ifr_equipped_cert', 'eng_no',
       'eng_type', 'eng_mfgr', 'hp_or_lbs', 'carb_fuel_injection',
       'eng_time_total', 'eng_time_last_insp', 'ntsb_no', 'ev_type', 'ev_date',
       'ev_dow', 'ev_time', 'ev_city', 'ev_state', 'ev_year', 'ev_month',
       'latitude', 'longitude', 'apt_name'],
      dtype='object')


In [18]:
# Listing metrics for all assigned columns 

for col in col_list:
  print(ntsb_1[col].name)
  col_metrics(ntsb_1, col) 

rwy_num
Unique data:  ['0' '14' '31' '20' '36' '18' '32' '27' '35' '16' '26r' '19' '28' '4r'
 '17' '34' '22' '28l' '15l' '11r' '17l' '13' '24' '10l' '9' '30r' '4' '15'
 '29' '26' '16r' '21r' '3' '9r' '25' '33' '1' '1r' '5r' '7' '12' '3r' '5'
 '30' '29r' '11' '21' '8' '10' '23' '19l' '2' '28r' '6' '36r' '26l' '14r'
 '17r' '19r' '25r' '27r' '35r' '12r' '6l' '8r' '22l' '9l' '2l' '18r' '1l'
 '32r' '36l' '34r' '29l' '31r' '22g' '27l' '20r' '11l' '8l' '22r' '34l'
 '6r' '16l' '4l' '23r' '5l' '18l' '20l' '7r' '13l' '13r' '31l' '25l' '24r'
 '23l' '24l' '35l' '21l' '30l' '12l' '20c' '7l' '33l' '30c' '36c' '32l'
 '23c' '17c' '3l' nan '3c' '2r' '33r' '15r' '10r' '360' '14l' '31c' '190'
 '9c' '250' '01' '02' '067' '09' '09r' '05' '2c' '310' '06' '0r' '00'
 '202' '08' '61' '04' '220' 'nw' '03' '315' '07' 'unk' '090' 'h1' '210'
 '160' '7u' '040' '080' '27u' '270' '180' '055' '350' '070' '030' '010'
 '01l' '050' 's' '09l' '5r7' '135' 'afo' 'gif' 'emm' '280' '110' '290'
 '2/8' '240' '320' '340' '020' '

Plans for columns: <br>
'rwy_num': Not sure... should we leave this? Or maybe join airport and runway number? <br>
'rwy_len': bucketize lengths<br>
'rwy_width': format number (currently scientific), then bucketize widths<br>
'ifr_equipped_cert': Good as is.<br>
'eng_no': Remove 448 nulls?<br>
'eng_type': Not sure what this is referring to.. need to spell these out when I figure it out... thinking 'elec' is electric.. <br>
'eng_mfgr': Consolidate engine manufacturers (i.e. gm and general motors)<br>
'hp_or_lbs': Change dtype from int64 to string, also, is this needed?<br>
'carb_fuel_injection': Change dtype from int64 to string <br>
'eng_time_total': bucketize hours<br>
'eng_time_last_insp': format number (currently scientific), then bucketize<br>
'ntsb_no': remove<br>
'ev_type': Change dtype from int64 to string <br>
'ev_date': convert dtype to datetime (date mm/dd/yyy)<br>
'ev_dow': Change dtype from int64 to string, make sure these are ordered categories<br>
'ev_time': format time dtype, take a look at bucketizing by hour<br>
'ev_city':  Change dtype from int64 to string <br>
'ev_state':  Change dtype from int64 to string, ensure all reflect a US state<br>
'ev_year': Remove? Unless an airport is more risky during different time periods which might be possible<br>
'ev_month': Ensure ordered list reflecting months <br>
'latitude': convert dtype to VARCHAR or DOUBLE. also maybe.... "You can use clustering algorithm like k-Nearest Neighbor algorithm to group your geo-location data (using a small number of potential clusters) and assign each cluster or a group a unique id. These unique id can then replace your latitude and longitude column."<br>
'longitude': see lat<br>
'apt_name': compare against airport list, consolidate private and private airstrip, and private strip


In [ ]:
# the columns to be used to create the labels are: inj_tot_t, damage, crew_inj_level, ev_highest_injury, damage

## ERICK'S SECITON

### CHI'S SECTION